In [1]:
#############################################################################
# regular express를 이용한 말뭉치 정재하기
#
#############################################################################
# 정규화할 인자들
alphabetP=True           # False=알바벳 제거
numberP=True             # False=숫자제거
punctuationP=False       # False=특수문자 (, . ? !) 제거
symbolP=False            # False=괄호((), [], {}, ], ')등 제거
repeat_numP=True        # False=반복되는 숫자+공백 제거 
remove_start_stringP=False     # False=해당 패턴으로 시작하는 문장 제거
remove_search_stringP=False     # False=해당 패턴 포함하는 문장 제거

# 4번이상 반복되는 단어 중복 제거 (remove_repeat_wordP=2 : 2번만 호출)
# => 예: 안녕안녕안녕안녕 => 안녕안녕, 안녕 안녕 안녕 안녕 => 그대로(*띄어쓰기한 단어는 중복제거 안됨)
remove_repeat_wordP=2     

# 4번이상 반복되는 문자 중복 제거 (nremove_repeat_charP=2 : 2번만 호출)
# => 예: aaaaa => aa, aaa aaa aaa aaa => 그대로(*띄어쓰기한 문자는 중복제거 안됨)
remove_repeat_charP=2

# 정규화할 입력 text
in_txt_path = '../../data11/my_corpus/kowiki-202206.txt'
#in_txt_path = '../../../korpora/moco/test-60.txt'

# 출력 text
out_txt_path = '../../data11/my_corpus/re-kowiki-202206.txt'

# 삭제할 최대 길이 문장
remove_min_len = 30 # 20이면 20보다 작은문장은 제거함

remove_max_len = 600 # 1000자 보다 큰 문장은 제거함

print_count = 100000 # 10000번째마다 문장 출력해봄

In [2]:
# reqular 함수들 정의 

# \d	숫자 [0-9]와 같다.
# \D	비숫자 [^0-9]와 같다.
# \w	숫자 + 문자 [a-zA-Z0-9]와 같다.
# \W	숫자 + 문자가 아닌 것 [^a-zA-Z0-9]와 같다.
# \s	공백 [ \t\n\r\f\v]와 같다.
# \S	비공백 [^ \t\n\r\f\v]와 같다.
# \b	단어 경계 (`\w`와 `\W`의 경계)
# \B	비단어 경계

import re

doublespace_pattern = re.compile('\s+')
repeatchars_pattern = re.compile('(\w)\\1{3,}')
number_pattern = re.compile('[0-9]')
punctuation_pattern = re.compile('[,\.\?\!]')
symbol_pattern = re.compile('[()\[\]\{\}`]')
hangle_pattern = re.compile('[ㄱ-ㅎㅏ-ㅣ가-힣]')
alphabet_pattern = re.compile('[a-zA-Z]')

hangle_filter = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣]')
hangle_number_filter = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9]')
text_filter = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9,\.\?\!\"\'-()\[\]\{\}]')

# 반복되는 숫자+띄어쓰기 패턴 제거 => 예: 12 13 14 11, 1 2 3 4 
repeat_number_pattern = [
    re.compile('\d\d\s\d\s'),
    re.compile('\d\s\d\d\s'),
    re.compile('\d\d\s\d\d\s'),
    re.compile('\d\s\d\s'),
]

# 아래 패턴으로 시작하는 문장은 제거함
# => 시작 문장이 아래 패턴이어야 함, 중간에 있는 경우 제거 안됨
# => match 사용
start_string_list = [
    re.compile("UPDATE ", re.IGNORECASE),
    re.compile("ALTER ", re.IGNORECASE),
    re.compile("CREATE ", re.IGNORECASE),  
    re.compile("IX A F", re.IGNORECASE), 
    re.compile("INPUTQM ", re.IGNORECASE), 
    re.compile('echo " ', re.IGNORECASE), 
    re.compile('MQCMD " ', re.IGNORECASE), 
    re.compile('COMMENT ON " ', re.IGNORECASE), 
    re.compile('FCFD'),
    re.compile('FFLD'),
    re.compile('ELSIF'),
    re.compile('ELEMENT ERR'),
    re.compile('BMSUSER IX'),
    re.compile('PK A ADJ'),
    re.compile('WMQ File'),
    re.compile('IBM SMART ANALYTICS'),
    re.compile('KHUB KIC'),  
    re.compile('V 0x'),   
    re.compile('PK A '),    
    re.compile('N4 A '),    
   
    
]

# 아래 패턴을 포함하는 문장은 제거함
# => search 사용
search_string_list = [
    re.compile("SQL SELECT ", re.IGNORECASE), # re.IGNORECASE = 대.소문자 구분 안함
    re.compile('SELECT COUNT ', re.IGNORECASE),
    re.compile("INSERT INTO ", re.IGNORECASE), 
    re.compile("SELECT FROM ", re.IGNORECASE),
    re.compile("SELECT LOG ", re.IGNORECASE),
    re.compile("SELECT STR ", re.IGNORECASE),
    re.compile("DELETE FROM ", re.IGNORECASE),
    re.compile('CString ', re.IGNORECASE), 
    re.compile('ULONG ', re.IGNORECASE), 
    re.compile('ULONGLONG " ', re.IGNORECASE), 
    re.compile('PUCHAR " ', re.IGNORECASE), 
    re.compile('VARCHAR ', re.IGNORECASE), 
    re.compile('ONLTEAISO ', re.IGNORECASE), 
    re.compile('EAI TRNS ', re.IGNORECASE), 
    re.compile('COMMENT ON ', re.IGNORECASE), 
    re.compile('DEFINE PROCESS ', re.IGNORECASE), 
    re.compile('EXEC SQL ', re.IGNORECASE), 
    re.compile('REPLACE DTL ', re.IGNORECASE), 
    re.compile(' Exception e ', re.IGNORECASE), 
    re.compile(' CASE WHEN ', re.IGNORECASE),  
    re.compile('public void ', re.IGNORECASE),  
    re.compile('DWORD ', re.IGNORECASE),  
    re.compile('DEBUG ', re.IGNORECASE),    
]

##========================================================================
# 4번이상 반복되는 단어 중복 제거 (num_repeat=2 : 2번만 호출)
# => 예: 안녕안녕안녕안녕 => 안녕안녕, 안녕 안녕 안녕 안녕 => 그대로(*띄어쓰기한 단어는 중복제거 안됨)
repeatchars_patterns = [
    re.compile('(\w\w\w\w)\\1{3,}'),
    re.compile('(\w\w\w)\\1{3,}'),
    re.compile('(\w\w)\\1{3,}'),
    re.compile('(\w)\\1{3,}')
]

def normalizetoken(sentence, num_repeat=2):
    tokens = sentence.split()
    return ' '.join(_normalize_korean_token(token, num_repeat) for token in tokens)

def _normalize_korean_token(token, num_repeat=2):
    #print(token)
    #token = _normalize_emoji(token)
    token = _remove_repeat(token, num_repeat)
    return token

def _remove_repeat(token, num_repeat=2):
    if num_repeat > 0:
        for pattern in repeatchars_patterns:
            #print(pattern)
            token = pattern.sub('\\1' * num_repeat, token)
    return token
##========================================================================


def normalize(sentence, 
              alphabet=False, 
              number=False,
              punctuation=False, 
              symbol=False, 
              repeat_num=False,
              remove_start_string=False,
              remove_search_string=False,
              remove_repeat_word=0, 
              remove_repeat_char=0):

    sentence = text_filter.sub(' ', sentence)
    
    # 패턴으로 시작하는 문장은 제거함
    # =>**만약 시작패턴이 아니라 중간에 있는 패턴도 제거하려면 match=>search 사용
    if not remove_start_string:
        for pattern in start_string_list:
            result = pattern.match(sentence)
            if result != None:
                 return None
    
    # 패턴이 포함된 문장은 제거함=>search 사용
    if not remove_search_string:
        for pattern in search_string_list:
            result = pattern.search(sentence)
            if result != None:
                 return None
                
    if not alphabet:
        sentence = alphabet_pattern.sub(' ', sentence)
        
    if not number:
        sentence = number_pattern.sub(' ', sentence)
        
    if not punctuation:
        sentence = punctuation_pattern.sub(' ', sentence)
        
    if not symbol:
        sentence = symbol_pattern.sub(' ', sentence)
        
    if not repeat_num:
        for num_pattern in repeat_number_pattern:
             sentence = num_pattern.sub(' ', sentence)
            
    if remove_repeat_char > 0:
        sentence = repeatchars_pattern.sub('\\1' * remove_repeat_char, sentence)
        
    if remove_repeat_word > 0:
        sentence = normalizetoken(sentence, num_repeat=remove_repeat_word)
        
    return doublespace_pattern.sub(' ', sentence).strip()

In [3]:
## yield 를 이용하여 generator 함수 정의    
def get_generator_corpus(data, max_len: int=100000):
    
    #dataset = data
    dataset = list(set(data)) # ****중복 문장 제거(*순서 유지 안함)
    
    for start_idx in range(0, len(dataset), max_len):
        samples = dataset[start_idx : start_idx + max_len]
        yield samples

In [4]:
from tqdm.notebook import tqdm

# 1.분석할 text 읽어오기(한줄씩 읽어오기)
with open(in_txt_path, 'r', encoding='utf8') as f:
    #lines = f.read()
    lines = f.readlines()
    #print(lines)
        
training_corpus = get_generator_corpus(lines)
        
result=[]
count = 0
for lines in tqdm(training_corpus):
    for line in tqdm(lines):
        
        #print(line)
        # false=제거함.
        clearline = normalize(line, 
                              alphabet=alphabetP, number=numberP, punctuation=punctuationP, 
                              symbol=symbolP, repeat_num=repeat_numP, remove_start_string=remove_start_stringP,
                              remove_search_string = remove_search_stringP,
                              remove_repeat_word=remove_repeat_wordP, remove_repeat_char=remove_repeat_charP)
        if clearline != None:
            clearline1=clearline.strip()
            count += 1
            if count % print_count == 0:  # print_count 째마다 출력해봄
                print(f'[{count}]{clearline1}')

            #if clearline1 not in result: # ****순서유지 중복 제거(*순서유지하는 경우 시간 오래걸림)

            # remove_min_len 보가 크고, remove_max_len 보다 작은 경우에만 저장
            if clearline1 != None:
                clearline1_len = len(clearline1)
                if clearline1_len > remove_min_len and clearline1_len < remove_max_len: # 20자 이상인 경우에만 출력
                    #print(clearline1)
                    result.append(clearline1)
    
# 중복 문장 제거(*순서 유지 안함)
#result = list(set(clearlinelist))

print('\r\n===[out]====')
print(f'*len:{len(result)}')
count = 0
for line1 in result:
    print(line1)
    count += 1
    if count > 10:
        break

0it [00:00, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

[100000]빌헬름 켐프는 첫 악장과 마지막 악장 모두를 위해 자신의 카덴자를 작곡했으며 이를 작품의 다양한 녹음에서 연주했다


  0%|          | 0/100000 [00:00<?, ?it/s]

...

[4100000]흥분성 세포 는 막 전위에 의해 작동하는 전압 개폐 나트륨 통로나 칼슘 통로를 가지고 있는 세포를 의미한다


  0%|          | 0/100000 [00:00<?, ?it/s]

[4200000]2m 오후나토시 아야사토구 를 뛰어넘는 높이로 야에야마 지진 당시 류큐 왕국 을 제외하면 일본에서 기록된 가장 높은 쓰나미이다


  0%|          | 0/13444 [00:00<?, ?it/s]

[4300000]Desperado 와 Under My Wheels Be My Lover 는 쿠퍼의 여러 컴필레이션 음반에 수록되었다

===[out]====
*len:3714432
스포츠 팀 문화 활동 해외 여행 등의 다양한 과외활동을 제공한다
남자 자유형 200m 종목의 예선기준 기록은 1분 47초 02이다
1785년 정조 9 당시 성균관 대사성 민종현 이 왕명에 의해 편찬한 태학지 건치 에 반궁도 가 목판화로 있는데 본 계첩과 30여 년의 시간적 차이가 있어서 그간에 변화된 성균관의 건물을 비교해 볼 수 있다
정지점 point of insertion 의 이름에 따라 명명된 세 부위로 나뉜다
DRM Digital Right Management 은 콘텐츠 생산자 제공자의 권리와 이익을 보호하기 위한 기술을 말한다
' 와 같은 것으로 언제나 확실하게 참인 것으로 판단되는 것을 말한다
이때 체코슬로바키아 바이마르 공화국 단치히 자유시 리투아니아 라트비아 루마니아 왕국 소련과 국경을 접했으며 그디니아를 통한 좁은 회랑으로 발트 해에 접하였다
매든은 754 승 705 패의 기록과 함께 자신의 재직 기간을 끝냈다
드론 고정익 전투기를 운용할 수 있으며 50 000톤으로 추정되는 만재배수량도 유사하다
그래서 주위에 있는 모든 레귤러 인물들이 이를 잘 알고 있다
이 영화는 2001년 칸 국제 영화제에서 첫 상영하였고 왓츠의 전미 비평가 협회 여우주연상 수상과 미국 영화 연구소 여우주연상 후보 지명등 많은 상과 지명을 받았다


In [5]:
# 리스트를 파일로 저장
with open(out_txt_path, 'w', encoding='utf8') as f:
    f.write('\n'.join(result))